## Transfer learning with PyTorch


In this tutorial you will learn how to configure and connect your own PyTorch model to Label Studio.

First you need to install pyheartex SDK that provides all endpoints for Label Studio to ML backend communication 

In [ ]:
! pip install pyheartex

Declare PyTorch data loaders that convert Label Studio's completions to neural network input feed

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import time

from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models, transforms

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


class ImageClassifierDataset(Dataset):
        
    def __init__(self, image_urls, image_classes):
        self.images = []
        self.labels = []
        
        self.classes = list(set(image_classes))
        self.class_to_label = {c: i for i, c in enumerate(self.classes)}
        
        self.image_size = 224
        self.transforms = transforms.Compose([
            transforms.Resize(self.image_size),
            transforms.CenterCrop(self.image_size),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        
        
        for image_url, image_class in zip(image_urls, image_classes):
            image = self._get_image_from_url(image_url)
            transformed_image = self.transforms(image)
            self.images.append(transformed_image)
            
            label = self.class_to_label[image_class]
            self.labels.append(label)
            
    def _get_image_from_url(self, url):
        pass
    
    def __getitem__(self, index):
        return self.images[index], self.labels[index]
    
    def __len__(self):
        return len(self.images)

Let's create a simple convolutional neural network image classifier based on pretrained ResNet18 model


In [ ]:
class ImageClassifier(object):
    
    def __init__(self, num_classes):
        self.model = models.resnet18(pretrained=True)
        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Linear(num_ftrs, num_classes)
        
        self.model = self.model.to(device)
        
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.SGD(self.model.parameters(), lr=0.001, momentum=0.9)
        
        # Decay LR by a factor of 0.1 every 7 epochs
        self.scheduler = optim.lr_scheduler.StepLR(self.optimizer, step_size=7, gamma=0.1)
    
    def save(self, path):
        torch.save(self.model.state_dict(), path)
    
    def load(self, path):
        self.model.load_state_dict(torch.load(path))
        self.model.eval()
        
    def train(self, dataloader, num_epochs=25):
        since = time.time()

        self.model.train()
        for epoch in range(num_epochs):
            print('Epoch {}/{}'.format(epoch, num_epochs - 1))
            print('-' * 10)
            
            running_loss = 0.0
            running_corrects = 0    
            # Iterate over data.
            for inputs, labels in dataloader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                self.optimizer.zero_grad()
                outputs = self.model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                self.scheduler.step()

            epoch_loss = running_loss / len(dataloader.dataset)
            epoch_acc = running_corrects.double() / len(dataloader.dataset)

            print('Train Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))

        print()
    
        time_elapsed = time.time() - since
        print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

        return self.model

Next step is to override API methods

In [ ]:
from htx.base_model import BaseModel


class ImageClassifierAPI(BaseModel):
    
    INPUT_TYPES = ('Image',)
    OUTPUT_TYPES = ('Choices',)
    
    def load(self, resources, **kwargs):
        """
        This method load model weights and any additional parameters into memory for further inference
        :param resources: dict-like object taken from training output
        :param kwargs: any additional parameters
        :return: 
        """
        self.model = ImageClassifier(resources['num_classes'])
        self.model.load(resources['model_path'])
        self.labels = resources['labels']
        
    def load_input_images(self, tasks):
        image_urls = [task['input'][0] for task in tasks]
        
    def predict(self, tasks, **kwargs):
        pass
        

Finally the training script is defined by a separate function that could lie in a separate environment. 
The only one convention is that it should consume task iterator as input and produce dict-like object of train resources used later by `load()` function.

In [ ]:
def train_image_classifier(input_tasks, output_dir, **kwargs):
    
    batch_size = 32
    num_epochs = 25     
    
    image_urls, image_classes = [], []
    for task in input_tasks:
        image_url = task['input'][0]
        image_class = task['output'][0]['value']['choices'][0]
        image_urls.append(image_url)
        image_classes.append(image_class)
        
    dataset = ImageClassifierDataset(image_urls, image_classes)
    dataloader = DataLoader(dataset, batch_size=batch_size)
    
    model = ImageClassifier(len(dataset.classes))
    model.train(dataloader, num_epochs)
    
    model.save(os.path.join(output_dir, 'model.pt'))


Now the model is ready, it's time to launch ML backend server.
First of all, let's create launch script called `wsgi.py`:

In [ ]:
import os
import argparse

from htx import app, init_model_server
from my_image_classifier import ImageClassifierAPI, train_image_classifier


init_model_server(
    create_model_func=ImageClassifierAPI,
    train_script=train_image_classifier,
    image_dir='~/.heartex/images',
    redis_queue=os.environ.get('RQ_QUEUE_NAME', 'default'),
    redis_host=os.environ.get('REDIS_HOST', 'localhost'),
    redis_port=os.environ.get('REDIS_PORT', 6379),
)


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument('--port', dest='port', default='9090')
    args = parser.parse_args()
    app.run(host='localhost', port=args.port, debug=True)
    

Now you have two possibilities depending on what scenario you'd expect from interacting with ML backend.

## Use Machine Learning backend only for prediction

If you want to run ML backend only for inference, i.e. getting the autolabels from pretrained model predictions, you can just launch the created script:

In [ ]:
! python wsgi.py

It automatically starts server on `http://localhost:9090`. To connect Label Studio app to the running server, initialize new project as the following:

In [ ]:
! label-studio start new_project --init --ml-backend-url http://localhost:9090 --template image_classification

That's all. Once you start labeling, you should be able to see the predicted image classes & ML backend server logs.

## Use Machine learning backend for prediction and retraining (active learning)
This scenario is suitable if you have already model trained to predict your target images, so that you can see prelabeling results and play with low-confident samples.
However if you want to keep your model constantly retraining after labeling some part of a dataset (which is a common case scenario with _active learning_), then you have to configure your server not only for inference but also for training.
Hopefully, pyheartex SDK provides a facility to do this without cumbersome coding by using RQ library. All you need is to start Redis server, RQ workers and link them to your `wsgi.py` via `RQ_QUEUE_NAME`, `REDIS_HOST`, `REDIS_PORT` parameters.

If you don't have time to launch RQ stack by yourself, you can leverage an off-the-shell docker configs. 
First make sure you have docker compose and docker installed on your system.
Then clone Git repo:

In [ ]:
! git clone https://github.com/heartexlabs/label-studio-ml-backend

 
Copy your files `wsgi.py` and `my_model.py` into the root directory `label-studio-ml-backend/`. Then build and run server:
 

In [ ]:
! docker-compose up

That's it. If everything is set, you can connect run Label Studio connected to your ML backend the same as above

In [ ]:
! label-studio start new_project --init --ml-backend-url http://localhost:9090 --template image_classification

Go to [Label Studio Machine Learning page](http://localhost:8080/ml.html) to check if everything is connected.